In [ ]:
from IPython.display import Image
import graphlab

Overview: Deploying the Dress Recommender
==================
Let's say you want to make an app which can recommend dresses to you based on a photo you took. 
You need a way to deploy the model previously built. Dato Predictive Services helps do this in an easy and scalable way. In this notebook, we demonstrate how do that for the dress recommender model.

<img src='images/predictive_services_overview.png'></img>

-----------------------


<img src="images/left.png"></img>
First, we define a function ```similar_dress``` that takes a query and returns similar dresses. This uses the same technique used in our previous session, in that we load a pretrained network to extract the features from the image, and then find similar items using the nearest neighbours model we created.

In [ ]:
pretrained_model = graphlab.load_model('../data/imagenet_model')

In [ ]:
nn_model = graphlab.load_model('../data/nearest_dress_model')

In [ ]:
def dress_similar(url):
    img = graphlab.Image(url)
    image_sf = graphlab.SFrame()
    image_sf['image'] = [img]
    image_sf['features'] = pretrained_model.extract_features(image_sf)
    ans = nn_model.query(image_sf, k=10)
    ans['image_url'] = ans['reference_label'].apply(lambda x: 'https://s3.amazonaws.com/gl-demo/multimedia_search/images/item_' + str(x) + '.jpg')
    return ans[['image_url']] 

In [ ]:
QUERY_URL = 'http://static.ddmcdn.com/gif/blue-dress.jpg'

In [ ]:
Image(QUERY_URL)

In [ ]:
ans = dress_similar(QUERY_URL)

Fetch the image from the response URL:

In [ ]:
ans['image'] = ans['image_url'].apply(lambda x: graphlab.Image(x))

In [ ]:
ans['image'].show()


<img src="images/middle.png"></img>
We add our ```similar_dress``` function to the predictive service, so that it can be queried by the outside world. Note: In order to follow along here, you need AWS credentials. You can then uncomment the code below. Be sure that the deployment state path is in an s3 bucket you have access to. 


In [ ]:
#gl.aws.set_credentials('AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY')

#ec2env = graphlab.deploy.Ec2Config()

#deployment = graphlab.deploy.predictive_service.create(
#                'ps-tutorial-dl',   # logical name
#                ec2env,        # EC2 config to use
#               's3://dato-developer-testing/similar-dresses') # deployment state path. You should change this to an S3
                                                               # bucket you have access to. 
#deployment = graphlab.deploy.predictive_service.load('s3://dato-developer-testing/similar-dresses')

#deployment.add('dress_similar', dress_similar)
#deployment.apply_changes()
#deployment.get_status()




The Predictive Service is now up! Let's query it and explore the response. 
<img src="images/right.png"></img>

In [ ]:
#response = deployment.query('dress_similar', url=QUERY_URL)

In [ ]:
response

Here, we're converting the response back to an SFrame:

In [ ]:
#sframe_response = graphlab.SFrame(response['response']).unpack('X1')

Here, we're fetching images from their url's. 

In [ ]:
#sframe_response['image'] = sframe_response['X1.image_url'].apply(lambda x: graphlab.Image(x))

Looking at the results

In [ ]:
#sframe_response.show()

RESTful Query
=======

Query from anywhere. Here, we issue a request via the ```requests``` library, and convert the response back into an SFrame. We then fetch the images from their url. This could easily be done from outside of Python, though. 

In [ ]:
def restful_query(url):
    import json
    import requests
    model_name = 'test'
    headers = {'content-type': 'application/json' }
    payload = {'api_key':'5b45e915-7942-4db7-b23e-243d5ce8cd9c', 'data':{'url':url}}
    end_point = 'http://%s/query/%s' % ('ps-tutorial-dl-573853473.us-west-2.elb.amazonaws.com','dress_similar' )
    resp = requests.post(end_point,json.dumps(payload), headers=headers)
    results = resp.json()
    return results


In [ ]:
response = restful_query('http://static.ddmcdn.com/gif/blue-dress.jpg')

Here, we convert the response into an SFrame. 

In [ ]:
sframe_response = graphlab.SFrame(response['response']).unpack('X1')

Here we fetch the image from their url's and visualize them.

In [ ]:
sframe_response['image'] = sframe_response['X1.image_url'].apply(lambda x: graphlab.Image(x))

In [ ]:
sframe_response.show()